In [13]:
import pandas as pd
import requests
from io import StringIO

requisicao = requests.get("https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2025_-_S%C3%A9rie_A")

html_content = StringIO(requisicao.text)
tabelas = pd.read_html(html_content)
tabela_classificacao = tabelas[7]
tabela_jogos = tabelas[8]
#display(tabela_classificacao)
#display(tabela_jogos)

In [14]:
nomes_times = list(tabela_jogos["Mandante \\ Visitante"])
apelidos = list(tabela_jogos.columns)
apelidos.pop(0)
#print(nomes_times)
#print(apelidos)

# Criando um dicionário para armazenar os dados
de_para_times = dict(zip(apelidos, nomes_times))
#print(de_para_times)


In [15]:

#print(de_para_times)
tabela_jogos_ajustaada  = tabela_jogos.set_index("Mandante \\ Visitante")
tabela_jogos_ajustaada= tabela_jogos_ajustaada.unstack().reset_index()
#print(tabela_jogos_ajustaada)
tabela_jogos_ajustaada = tabela_jogos_ajustaada.rename(columns={"level_0": "Visitante", "Mandante \\ Visitante": "Mandante", 0: "resultado"})

print(tabela_jogos_ajustaada.columns)

# Ajuste o apelido do time
def ajustar_apelido_time(linha):
    apelido = linha["Visitante"]  
    nome = de_para_times.get(apelido, apelido)
    return nome

tabela_jogos_ajustaada["Visitante"] = tabela_jogos_ajustaada.apply(ajustar_apelido_time, axis=1)
tabela_jogos_ajustaada = tabela_jogos_ajustaada[tabela_jogos_ajustaada["Visitante"] != tabela_jogos_ajustaada["Mandante"]]
# Verifique os resultados   
#display(tabela_jogos_ajustaada)


Index(['Visitante', 'Mandante', 'resultado'], dtype='object')


In [16]:
tabela_jogos_ajustaada["resultado"] = tabela_jogos_ajustaada["resultado"].fillna("a jogar")
tabela_jogos_realizados = tabela_jogos_ajustaada[tabela_jogos_ajustaada["resultado"].str.contains("–")]
tabela_jogos_faltantes = tabela_jogos_ajustaada[~tabela_jogos_ajustaada["resultado"].str.contains("–")]
tabela_jogos_faltantes = tabela_jogos_faltantes.drop(columns=["resultado"])

#display(tabela_jogos_realizados)

In [ ]:
tabela_jogos_realizados[["gols_casa", "gols_fora"]] = tabela_jogos_realizados["resultado"].str.split("–", expand=True)
tabela_jogos_realizados = tabela_jogos_realizados.drop(columns=["resultado"])
tabela_jogos_realizados["gols_casa"] = tabela_jogos_realizados["gols_casa"].astype(int)
tabela_jogos_realizados["gols_fora"] = tabela_jogos_realizados["gols_fora"].astype(int)
#display(tabela_jogos_realizados)


In [18]:
media_gols_casa = tabela_jogos_realizados.groupby("Mandante").mean(numeric_only=True)
media_gols_casa = media_gols_casa.rename(columns={"gols_casa": "gols feitos casa", "gols_fora": "gols sofridos casa"})   
media_gols_fora = tabela_jogos_realizados.groupby("Visitante").mean(numeric_only=True)
media_gols_fora = media_gols_fora.rename(columns={"gols_casa": "gols sofridos fora", "gols_fora": "gols feitos fora"})
#display(media_gols_fora)

In [19]:
tabela_estatisticas = media_gols_casa.merge(media_gols_fora, left_index=True, right_index=True)
tabela_estatisticas = tabela_estatisticas.reset_index()
tabela_estatisticas = tabela_estatisticas.rename(columns={"Mandante": "time"})
#display(tabela_estatisticas)


In [20]:
from scipy.stats import poisson

# como a partir das medias calcular a probabilidade do time ganhar/perder/empate

def calcular_pontuacao_esperada(linha):
    time_casa = linha["Mandante"]
    time_fora = linha["Visitante"]

    # qual a média de gols que o Cruzeiro, jogando em casa, faz contra o Vasco, jogando fora de casa
    lambda_casa = (tabela_estatisticas.loc[tabela_estatisticas["time"]==time_casa, "gols feitos casa"].iloc[0] 
                * tabela_estatisticas.loc[tabela_estatisticas["time"]==time_fora, "gols sofridos fora"].iloc[0])
    # média de gols cruzeiro dentro de casa * média de gols sofridos pelo vasco fora de casa


    # média de gols que o vasco faz fora de casa * media de gols que o Cruzeiro sofre dentro de casa
    lambda_fora =  (tabela_estatisticas.loc[tabela_estatisticas["time"]==time_fora, "gols feitos fora"].iloc[0] 
                    * tabela_estatisticas.loc[tabela_estatisticas["time"]==time_casa, "gols sofridos casa"].iloc[0])




    pv_casa = 0
    p_empate = 0
    pv_fora = 0

    for gols_casa in range(8):
        for gols_fora in range(8):
            probabilidade_resultado = poisson.pmf(gols_casa, lambda_casa) * poisson.pmf(gols_fora, lambda_fora)
            if gols_casa == gols_fora:
                p_empate += probabilidade_resultado
            elif gols_casa > gols_fora:
                pv_casa += probabilidade_resultado
            elif gols_casa < gols_fora:
                pv_fora += probabilidade_resultado

    # print(pv_casa)
    # print(p_empate)
    # print(pv_fora)

    ve_casa = pv_casa * 3 + p_empate
    ve_fora = pv_fora * 3 + p_empate
    linha["pontos_casa"] = ve_casa
    linha["pontos_fora"] = ve_fora
    return linha

# como a partir da probabilidade você calcula a pontuação que o time vai ter
# ValorEsperado - pontuação do vasco

# Vasco x Cruzeiro

# p_vasco = 50% * 3 + 10% * 1
# p_cruzeiro = 40% * 3 + 10% * 1

# 50% * 3 = 1.8
# 10%
# 40% * 3 = 1.2

In [21]:
tabela_jogos_faltantes = tabela_jogos_faltantes.apply(calcular_pontuacao_esperada, axis=1)
#display(tabela_jogos_faltantes)

In [22]:
def ajustar_nome_time(linha):
    for nome in nomes_times:
        if nome in linha["Equipevde"]:
            return nome
    
tabela_classificacao["time"] = tabela_classificacao.apply(ajustar_nome_time, axis=1)
tabela_classificacao_atualizada = tabela_classificacao[["time", "Pts"]]
tabela_classificacao_atualizada["Pts"] = tabela_classificacao_atualizada["Pts"].astype(int)

#display(tabela_classificacao_atualizada)

C:\Users\macha\AppData\Local\Temp\ipykernel_8696\3656241227.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabela_classificacao_atualizada["Pts"] = tabela_classificacao_atualizada["Pts"].astype(int)


In [23]:
tabela_pontuacao_casa = tabela_jogos_faltantes.groupby("Mandante").sum(numeric_only=True)[["pontos_casa"]]
tabela_pontuacao_fora = tabela_jogos_faltantes.groupby("Visitante").sum(numeric_only=True)[["pontos_fora"]]
#display(tabela_pontuacao_casa)
#display(tabela_pontuacao_fora)

In [24]:
def atualizar_pontuacao_previsao(linha):
    time = linha["time"]
    pontuacao = int(linha["Pts"]) + float(tabela_pontuacao_casa.loc[time, "pontos_casa"]) + float(tabela_pontuacao_fora.loc[time, "pontos_fora"])
    return pontuacao


tabela_classificacao_atualizada["Pts"] = tabela_classificacao_atualizada.apply(atualizar_pontuacao_previsao, axis=1)
tabela_classificacao_atualizada = tabela_classificacao_atualizada.sort_values(by="Pts", ascending=False)
tabela_classificacao_atualizada = tabela_classificacao_atualizada.reset_index(drop=True)
tabela_classificacao_atualizada.index = tabela_classificacao_atualizada.index + 1
display(tabela_classificacao_atualizada)

C:\Users\macha\AppData\Local\Temp\ipykernel_8696\3357124533.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabela_classificacao_atualizada["Pts"] = tabela_classificacao_atualizada.apply(atualizar_pontuacao_previsao, axis=1)


,time,Pts
1,Palmeiras,72.180845
2,Flamengo,70.364909
3,Red Bull Bragantino,66.972181
4,Internacional,59.196179
5,Fluminense,55.875924
6,Ceará,55.396993
7,Cruzeiro,54.497121
8,São Paulo,52.783903
9,Botafogo,52.721402
10,Bahia,52.433548
